In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("archive/winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
df.shape

(1599, 12)

In [4]:
input_cols = list(df.columns)[:-1]
input_cols

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [5]:
output_cols = ['quality']

In [6]:
def dataframe_to_arrays(df):
    # Make a copy of the original dataframe
    df1 = df.copy(deep=True)
    # Extract input & outupts as numpy arrays
    inputs_array = df1[input_cols].to_numpy()
    targets_array = df1[output_cols].to_numpy()
    return inputs_array, targets_array

In [7]:
inputs_array, targets_array = dataframe_to_arrays(df)
inputs_array, targets_array

(array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
        [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
        [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
        ...,
        [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
        [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
        [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]]),
 array([[5],
        [5],
        [5],
        ...,
        [6],
        [5],
        [6]]))

In [8]:
inputs_array.shape,targets_array.shape

((1599, 11), (1599, 1))

In [9]:
import torch
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

In [10]:
from torch.utils.data import DataLoader, TensorDataset, random_split
dataset = TensorDataset(inputs, targets)

In [11]:
df.shape

(1599, 12)

In [12]:
num_rows = len(df)
val_percent = 0.8 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_df, val_df = random_split(dataset, [train_size, val_size]) 

In [13]:
batch_size = 50

In [14]:
train_loader = DataLoader(train_df, batch_size, shuffle=True)
val_loader = DataLoader(val_df, batch_size)

In [15]:
input_size = len(input_cols)
output_size = len(output_cols)

In [16]:
import torch.nn as nn
import torch.nn.functional as F
class WineModel(nn.Module):
    def __init__(self,number_of_neurons):
        super().__init__()     
        self.fc1 = nn.Linear(input_size, number_of_neurons)
        self.fc2 = nn.Linear(number_of_neurons,output_size)
        # fill this (hint: use input_size & output_size defined above)
        #model initialized with random weight
        
    def forward(self, xb):
        xb = F.relu(self.fc1(xb))
        out = self.fc2(xb)             # batch wise forwarding
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)         
        # Calcuate loss
        loss = F.l1_loss(out, targets)  # batch wise training step and loss
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss =F.l1_loss(out, targets)       # batch wise validation and loss    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine val losses of all batches as average
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 500 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [17]:
model =  WineModel(4)

In [18]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0575, -0.1271,  0.2574, -0.1277,  0.2486,  0.2220, -0.2098, -0.0976,
          -0.1612,  0.1560, -0.1551],
         [ 0.1499, -0.1783,  0.1082,  0.2664,  0.2110,  0.2729,  0.1988,  0.1660,
          -0.2869, -0.1021, -0.0189],
         [-0.2989, -0.2046, -0.1111,  0.2616, -0.1147, -0.2715,  0.2243,  0.0385,
          -0.2194,  0.0686,  0.0579],
         [ 0.0597,  0.2914,  0.1948, -0.0487,  0.2892,  0.1351,  0.2762, -0.0745,
          -0.2176, -0.1808,  0.2011]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1776, -0.1106,  0.0685,  0.1214], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1002, -0.3663,  0.4754, -0.0335]], requires_grad=True),
 Parameter containing:
 tensor([0.3736], requires_grad=True)]

In [19]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)  #appends total validation loss of whole validation set epoch wise
    return history

In [20]:
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url

In [21]:
result = evaluate(model,val_loader) # Use the the evaluate function
print(result)

{'val_loss': 9.000547409057617}


In [22]:
result['val_loss']

9.000547409057617

In [23]:
#epochs = 1000
#lr = 1e-2
#history1 = fit(epochs, lr, model, train_loader, val_loader)

In [24]:
#epochs = 1000
#lr = 1e-3
#history2 = fit(epochs, lr, model, train_loader, val_loader)

In [25]:
val_loss = evaluate(model,val_loader)
val_loss

{'val_loss': 9.000547409057617}

In [26]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0) 
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [27]:
input, target = val_df[0]
predict_single(input, target, model)

Input: tensor([ 8.7000,  0.6900,  0.3100,  3.0000,  0.0860, 23.0000, 81.0000,  1.0002,
         3.4800,  0.7400, 11.6000])
Target: tensor([6.])
Prediction: tensor([-4.2943])


In [28]:
input, target = val_df[10]
predict_single(input, target, model)

Input: tensor([10.4000,  0.6100,  0.4900,  2.1000,  0.2000,  5.0000, 16.0000,  0.9994,
         3.1600,  0.6300,  8.4000])
Target: tensor([3.])
Prediction: tensor([-1.9000])


In [29]:
#input, target = val_df[5]
#predict_single(input, target, model)

In [30]:
class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

    
        self.velocity_i.append(random.uniform(-1,1))
        self.position_i.append(x0)

    # evaluate current fitness
    def evaluate(self):
    	#self.err_i=costFunc(self.position_i)
        #net = Net(int(round(self.position_i[0])))
        net = WineModel(round(self.position_i[0]))
    	#print(net)
        #optimizer = optim.Adam(net.parameters(), lr=0.05)
        #def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
        #history = []
        lr = 1e-2
        epochs = 2000
        optimizer = torch.optim.SGD(net.parameters(), lr)
        for epoch in range(epochs):
            # Training Phase 
            for batch in train_loader:
                loss = net.training_step(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            # Validation phase
            result = evaluate(net, val_loader)
            net.epoch_end(epoch, result, epochs)
            #history.append(result)  #appends total validation loss of whole validation set epoch wise
        self.err_i = result['val_loss']

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        r1=random.random()
        r2=random.random()

        vel_cognitive=c1*r1*(self.pos_best_i[0]-self.position_i[0])
        vel_social=c2*r2*(pos_best_g[0]-self.position_i[0])
        self.velocity_i[0]=w*self.velocity_i[0]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        self.position_i[0]=self.position_i[0]+self.velocity_i[0]

        # adjust maximum position if necessary
        if self.position_i[0]>bounds[1]:
        	self.position_i[0]=bounds[1]

        # adjust minimum position if neseccary
        if self.position_i[0] < bounds[0]:
        	self.position_i[0]=bounds[0]

In [31]:
import math
import random

class PSO():
    def __init__(self,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions= 1
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            x0 = random.randint(bounds[0],bounds[1])
            print('Particle ',i,' position is ', x0)
            swarm.append(Particle(x0))
        #print('Swarm : ',swarm)

        # begin optimization loop
        i=0
        while i < maxiter:
            print('Iteration :', i)
            print('*****************************')
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                print('Particle: ',j)
                swarm[j].evaluate()
                #print('swarm[{}].evaluate = {}'.format(j, swarm[j].evaluate(costFunc)))

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)
                print('pos_best_g so far', pos_best_g)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print('FINAL:')
        print(pos_best_g)
        print(err_best_g)

if __name__ == "__main__":

#--- RUN ----------------------------------------------------------------------+

    #initial = 5               # initial starting location [x1,x2...]
    bounds=[1,10]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
    PSO(bounds,num_particles=7,maxiter=10)

Particle  0  position is  5
Particle  1  position is  6
Particle  2  position is  6
Particle  3  position is  1
Particle  4  position is  9
Particle  5  position is  2
Particle  6  position is  2
Iteration : 0
*****************************
Particle:  0
Epoch [500], val_loss: 0.6681
Epoch [1000], val_loss: 0.5808
Epoch [1500], val_loss: 0.5614
Epoch [2000], val_loss: 0.5617
pos_best_g so far [5]
Particle:  1
Epoch [500], val_loss: 0.5687
Epoch [1000], val_loss: 0.5652
Epoch [1500], val_loss: 0.5716
Epoch [2000], val_loss: 0.7875
pos_best_g so far [5]
Particle:  2
Epoch [500], val_loss: 0.5779
Epoch [1000], val_loss: 0.5819
Epoch [1500], val_loss: 0.5972
Epoch [2000], val_loss: 0.5562
pos_best_g so far [6]
Particle:  3
Epoch [500], val_loss: 0.6635
Epoch [1000], val_loss: 0.6630
Epoch [1500], val_loss: 0.6630
Epoch [2000], val_loss: 0.6633
pos_best_g so far [6]
Particle:  4
Epoch [500], val_loss: 0.5923
Epoch [1000], val_loss: 0.6072
Epoch [1500], val_loss: 0.6255
Epoch [2000], val_loss: